In [ ]:
import sys
sys.path.append('../..')
from backtester import matlab, backtester
from backtester.analysis import *
import pandas as pd
import numpy as np
import glob, os

import itertools

from pandas.tseries.offsets import BDay

from backtester.exoinfo import EXOInfo
from exobuilder.data.exostorage import EXOStorage
from IPython.core.display import display, HTML



import pandas as pd
import numpy as np
import scipy
import pprint

%pylab inline
figsize(20,10)

In [ ]:
# Only Cont Fut Flag
only_cont_fut = False

In [ ]:
# Loading global setting for MongoDB etc.
from scripts.settings import *

try:
    from scripts.settings_local import *
except:
    pass

storage = EXOStorage(MONGO_CONNSTR, MONGO_EXO_DB)
pp = pprint.PrettyPrinter(indent=4)

exo_list = storage.exo_list()
pp.pprint(exo_list)


df = pd.DataFrame()

for e in exo_list:
    exo_series, exo_dict = storage.load_series(e)
    if only_cont_fut == True:   
        if 'Fut' in exo_dict['name']:   
            exo_series = exo_series[exo_series.columns[0]]
            df[exo_dict['name']] = exo_series
            
    if only_cont_fut == False:
        exo_series = exo_series[exo_series.columns[0]]
        df[exo_dict['name']] = exo_series
#exo_series.plot()

In [ ]:
df.head()

In [ ]:
px_ser = df.ES_CallSpread.dropna()

In [ ]:
'''
https://ru.tradingview.com/stock-charts-support/index.php/Directional_Movement_(DMI)

Calculating the DMI can actually be broken down into two parts. 
First, calculating the +DI and -DI, and second, calculating the ADX.
 
To calculate the +DI and -DI you need to find the +DM and -DM (Directional Movement). 
+DM and -DM are calculated using the High, Low and Close for each period. 
You can then calculate the following:
 
Current High - Previous High = UpMove
Current Low - Previous Low = DownMove
 
If UpMove > DownMove and UpMove > 0, then +DM = UpMove, else +DM = 0
If DownMove > Upmove and Downmove > 0, then -DM = DownMove, else -DM = 0
 
Once you have the current +DM and -DM calculated, the +DM and -DM lines can be 
calculated and plotted based on the number of user defined periods.
 
+DI = 100 times Exponential Moving Average of (+DM / Average True Range)
-DI = 100 times Exponential Moving Average of (-DM / Average True Range)
 
Now that -+DX and -DX have been calculated, the last step is calculating the ADX.
 
ADX = 100 times the Exponential Moving Average of the Absolute Value of (+DI- -DI) / (+DI + -DI)'''

hl_data_available = False

C = px_ser

if hl_data_available == False:
    
    H = C.rolling(2).max().dropna()
    L = C.rolling(2).min().dropna()

    C = C[H.index]

prev_high = H.shift(1)
prev_low = L.shift(1)

dm_pos = pd.Series(0.0, index=C.index)
dm_neg = pd.Series(0.0, index=C.index)

atr = ATR(H,L,C,14)

for i in range(C.size):
    upmove = H[i] - prev_high[i]
    downmove =  prev_low[i] - L[i]
    #print(upmove > downmove)
    
    if (upmove > downmove) & (upmove > 0):
        dm_pos[i] = upmove
        #print(dm_pos.iloc[i])
        
    if (downmove > upmove) & (downmove > 0):
        dm_neg[i] = downmove

di_pos = ((dm_pos.ewm(14).mean() / atr).ewm(14).mean()) * 100
di_neg = ((dm_neg.ewm(14).mean() / atr).ewm(14).mean()) * 100

adx = ((di_pos - di_neg) / (di_pos + di_neg)).abs().ewm(20).mean() * 100

In [ ]:
px_ser.ix['2012':].plot()
#px_ser.rolling(2).max().ix['2016'].plot()
#px_ser.rolling(2).min().ix['2016'].plot()

di_pos.ix['2012':].plot(secondary_y=True)
di_neg.ix['2012':].plot(secondary_y=True)
#((di_pos + di_neg)/2).ix['2012':].plot(secondary_y=True)
#adx.ix['2012':].plot(secondary_y=True)

In [ ]:
px_ser.ix['2012':].plot()

adx.ix['2012':].plot(secondary_y=True)

# Entry rules

In [ ]:
px_ser.plot()

# Difference of DIs is lower/higher than some levels (10-20-40)
(di_pos - di_neg).plot(secondary_y=True)

axhline(40)
axhline(-40)

axhline(20)
axhline(-20)

In [ ]:
px_ser.plot()


#di_pos.ix['2012':].plot(secondary_y=True)
#di_neg.ix['2012':].plot(secondary_y=True)

adx.plot(secondary_y=True)
adx.expanding(60).quantile(0.2).plot(secondary_y=True)
adx.expanding(60).quantile(0.5).plot(secondary_y=True)
adx.expanding(60).quantile(0.8).plot(secondary_y=True)

In [ ]:
px_ser.plot()

adx.pct_change(10).rank(pct=True).plot()

di_pos.ix['2012':].plot(secondary_y=True)
di_neg.ix['2012':].plot(secondary_y=True)

In [ ]:

di_pos.plot()

di_pos.expanding(60).quantile(0.1).plot()
di_pos.expanding(60).quantile(0.5).plot()
di_pos.expanding(60).quantile(0.9).plot()

px_ser.plot(secondary_y=True)

In [ ]:
di_pos.rank(pct=True).plot()